### <Center> Лабораторна робота №7. <br> Алгоритм випадкового лісу в задачі кредитного скорінгу

Необхідно розв'язати задачу кредитного скорінга 

Ознаки клієнта банку:
- Age - вік (дійсночислова)
- Income - місячний дохід (дійсночислова)
- BalanceToCreditLimit - відношення балансу на кредитній картці до ліміту за кредитом (дійсночислова)
- DIR - Debt-to-income Ratio (дійсночислова)
- NumLoans - кылькість позичок і кредитних ліній
- NumRealEstateLoans - кількість іпотек і позичок, пов'язаних з нерухомістю (натуральне число)
- NumDependents - кількість членів сім'ї, яких утримує клієнт, без врахування самого клієнту (натуральне число)
- Num30-59Delinquencies - кількість протермінувань виплат за кредитом від 30 до 59 днів (натуральне число)
- Num60-89Delinquencies - кількість протермінувань виплат за кредитом від 60 до 89 дній (натуральне число)
- Delinquent90 - чи були протермінування виплат за кредитом більше 90 днів (бінарний)

In [17]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
%matplotlib inline

**Спочатку налаштуємо доступ до даних на google drive (якщо ви відкриваєте блокнот в google colab, а не на PC) шляхом монтування google drive**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive') 

Перевіримо шлях до папки з матеріалами лаборатоної роботи на google drive. Якщо у вас шлях відрізняється то відредагуйте

In [ ]:
!ls gdrive/'My Drive'/TEACHING/IntroDataScience/intro_to_data_science/Lab_7_8/data

Перемістимо матеріали лабораторної роботи з google drive на віртуальну машину google colab

In [ ]:
!cp -a gdrive/'My Drive'/TEACHING/IntroDataScience/intro_to_data_science/Lab_7_8/. .
!ls

Завантажимо дані з використанням pandas

In [18]:
train_df = pd.read_csv('data/credit_scoring_train.csv', index_col='client_id')
test_df = pd.read_csv('data/credit_scoring_test.csv', index_col='client_id')

In [19]:
y = train_df['Delinquent90']
train_df.drop('Delinquent90', axis=1, inplace=True)

In [20]:
train_df.head()

,DIR,Age,NumLoans,NumRealEstateLoans,NumDependents,Num30-59Delinquencies,Num60-89Delinquencies,Income,BalanceToCreditLimit
client_id,,,,,,,,,
0,0.496289,49.1,13,0,0.0,2,0,5298.360639,0.387028
1,0.433567,48.0,9,2,2.0,1,0,6008.056256,0.234679
2,2206.731199,55.5,21,1,NaN,1,0,NaN,0.348227
3,886.132793,55.3,3,0,0.0,0,0,NaN,0.971930
4,0.000000,52.3,1,0,0.0,0,0,2504.613105,1.004350


**Переглянемо кількість пропусків в кожній ознаці.**

In [26]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 75000 entries, 0 to 74999
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   DIR                    75000 non-null  float64
 1   Age                    75000 non-null  float64
 2   NumLoans               75000 non-null  int64  
 3   NumRealEstateLoans     75000 non-null  int64  
 4   NumDependents          75000 non-null  float64
 5   Num30-59Delinquencies  75000 non-null  int64  
 6   Num60-89Delinquencies  75000 non-null  int64  
 7   Income                 75000 non-null  float64
 8   BalanceToCreditLimit   75000 non-null  float64
dtypes: float64(5), int64(4)
memory usage: 5.7 MB


In [27]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 75000 entries, 75000 to 149999
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   DIR                    75000 non-null  float64
 1   Age                    75000 non-null  float64
 2   NumLoans               75000 non-null  int64  
 3   NumRealEstateLoans     75000 non-null  int64  
 4   NumDependents          75000 non-null  float64
 5   Num30-59Delinquencies  75000 non-null  int64  
 6   Num60-89Delinquencies  75000 non-null  int64  
 7   Income                 75000 non-null  float64
 8   BalanceToCreditLimit   75000 non-null  float64
dtypes: float64(5), int64(4)
memory usage: 5.7 MB


**Замінимо пропуски медіанними значеннями.**

In [28]:
train_df['NumDependents'].fillna(train_df['NumDependents'].median(), inplace=True)
train_df['Income'].fillna(train_df['Income'].median(), inplace=True)
test_df['NumDependents'].fillna(test_df['NumDependents'].median(), inplace=True)
test_df['Income'].fillna(test_df['Income'].median(), inplace=True)

### Дерево рішень без налаштування параметрів

**Обучите дерево решений максимальной глубины 3, используйте параметр random_state=17 для воспроизводимости результатов.**

In [31]:
first_tree = DecisionTreeClassifier(max_depth=3, random_state=17)
first_tree.fit(train_df, y)

DecisionTreeClassifier(max_depth=3, random_state=17)

**Зробіть прогноз для тестової вибірки.**

In [39]:
first_tree_pred = first_tree.predict(test_df)

**Запишемо прогноз у файл.**

In [45]:
def write_to_submission_file(predicted_labels, out_file,
                             target='Delinquent90', index_label="client_id"):
    # turn predictions into data frame and save as csv file
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(75000, 
                                                  predicted_labels.shape[0] + 75000),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

In [46]:
write_to_submission_file(first_tree_pred, 'credit_scoring_first_tree_v1.csv')

**Якщо прогнозувати ймовірності дефолту для клієнтів тестової вибірки, результат буде набагато кращим.**

In [47]:
first_tree_pred_probs = first_tree.predict_proba(test_df)[:, 1]

In [48]:
write_to_submission_file(first_tree_pred_probs, 'credit_scoring_first_tree_v2.csv')

## Дерево рішень без налаштування параметрів за допомогою GridSearch

**Налаштуйте параметри дерева за допомогою `GridSearhCV`, подивіться на кращу комбінацію параметрів і середню якість на 5-кратній крос-валідації. Використовуйте параметр `random_state=17` (для відтворюваності результатів), не забувайте про розпаралелювання (`n_jobs=-1`).**

In [49]:
tree_params = {'max_depth': list(range(3, 8)), 
               'min_samples_leaf': list(range(5, 13))}

locally_best_tree = GridSearchCV(first_tree, tree_params, cv=5, n_jobs=-1)
locally_best_tree.fit(train_df, y)

GridSearchCV(cv=5,
             estimator=DecisionTreeClassifier(max_depth=3, random_state=17),
             n_jobs=-1,
             param_grid={'max_depth': [3, 4, 5, 6, 7],
                         'min_samples_leaf': [5, 6, 7, 8, 9, 10, 11, 12]})

In [50]:
locally_best_tree.best_params_, round(locally_best_tree.best_score_, 3)

({'max_depth': 5, 'min_samples_leaf': 11}, 0.935)

**Зробіть прогноз для тестової вибірки.**

In [51]:
tuned_tree_pred_probs = locally_best_tree.predict_proba(test_df)[:, 1]

In [52]:
write_to_submission_file(tuned_tree_pred_probs, 'credit_scoring_first_tree_v3.csv')

### Випадковий ліс без настройки параметрів

**Навчіть випадковий ліс з дерев необмеженої глибини, використовуйте параметр `random_state=17` для відтворюваності результатів.**

In [53]:
first_forest = RandomForestClassifier(random_state=17)
first_forest.fit(train_df, y)

RandomForestClassifier(random_state=17)

In [54]:
first_forest_pred = first_forest.predict_proba(test_df)[:, 1]

**Зробіть прогноз для тестової вибірки.**

In [55]:
write_to_submission_file(first_forest_pred, 'credit_scoring_first_tree_v4.csv')

### Випадковий ліс з налаштуванням параметрів

**Налаштуйте параметр `max_features` лісу за допомогою `GridSearhCV`, подивіться на кращу комбінацію параметрів і середню якість на 5-кратній крос-валідації. Використовуйте параметр random_state=17 (для відтворюваності результатів), не забувайте про розпаралелювання (n_jobs=-1).**

In [56]:
%%time
forest_params = {'max_features': np.linspace(.3, 1, 7)}

locally_best_forest = GridSearchCV(first_forest, forest_params, cv=5, n_jobs=-1)
locally_best_forest.fit(train_df, y)

Wall time: 4min 36s


GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=17), n_jobs=-1,
             param_grid={'max_features': array([0.3       , 0.41666667, 0.53333333, 0.65      , 0.76666667,
       0.88333333, 1.        ])})

In [57]:
locally_best_forest.best_params_, round(locally_best_forest.best_score_, 3)

({'max_features': 0.3}, 0.934)

In [58]:
tuned_forest_pred = locally_best_forest.predict_proba(test_df)[:, 1]

In [59]:
write_to_submission_file(tuned_forest_pred, 'credit_scoring_first_tree_v5.csv')

**Подивіться, як налаштований випадковий ліс оцінює важливість ознак за їх впливом на цільову ознаку. Подайте результати в наглядному вигляді за допомогою `DataFrame`.**

In [60]:
pd.DataFrame(locally_best_forest.best_estimator_.feature_importances_)

,0
0,0.170093
1,0.160560
2,0.093227
3,0.032962
4,0.038709
5,0.063074
6,0.065174
7,0.150816
8,0.225384


**Як правило збільшення кількості дерев тільки покращує результат. Так що на останок навчіть випадковий ліс з 300 дерев зі знайденими кращими параметрами. Це може зайняти декілька хвилин.**

In [61]:
%%time
final_forest = RandomForestClassifier(n_estimators=300, random_state=17, max_features=0.3, n_jobs=-1)
final_forest.fit(train_df, y)
final_forest_pred = final_forest.predict_proba(test_df)[:, 1]
write_to_submission_file(final_forest_pred, 'credit_scoring_final_forest.csv')

Wall time: 15 s
